In [1]:
import os
import h5py

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import astropy.coordinates as coord

%matplotlib inline
plt.style.use('/jet/home/tvnguyen/mplstyle/default.mplstyle')

In [2]:
plotdir = '/jet/home/tvnguyen/accreted_catalog/gaia_accreted_catalog/notebooks/figures'

In [4]:
catalog_root = '/ocean/projects/phy210068p/shared/gaia_catalogs/'
catalog_name = 'GaiaDR3_reduced_v1'
features = ('ra', 'dec', 'pmra', 'pmdec', 'parallax', 'radial_velocity')

catalog = {k: [] for k in features}
catalog['score'] = []
catalog['index'] = []
for i in range(10):
    data_path = os.path.join(catalog_root, catalog_name, f'data.{i}.hdf5')
    labels_path = os.path.join(catalog_root, catalog_name, f'labels.{i}.hdf5')
    print(f'Loading data from {data_path} and {labels_path}')

    with h5py.File(labels_path, 'r') as f:
        score = f['score'][:]
        label_source_id = f['source_id'][:]
        index = f['index'][:]
    catalog['score'].append(score)
    catalog['index'].append(index)

    with h5py.File(data_path, 'r') as f:
        for k in features:
            catalog[k].append(f[k][:][index])  # not f[k][index] because it is slower for some reason

for key in catalog:
    catalog[key] = np.concatenate(catalog[key], axis=0)

Loading data from /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/data.0.hdf5 and /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/labels.0.hdf5
Loading data from /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/data.1.hdf5 and /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/labels.1.hdf5
Loading data from /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/data.2.hdf5 and /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/labels.2.hdf5
Loading data from /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/data.3.hdf5 and /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/labels.3.hdf5
Loading data from /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/data.4.hdf5 and /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/labels.4.hdf5
Loading data from /ocean/projects/phy210068p/shared/gaia_catalogs/GaiaDR3_reduced_v1/data.

In [15]:
# convert from RA, Dec, parallax, pmra, pmdec, radial_velocity to Galactic coordinates
distance = coord.Distance(parallax=catalog['parallax']*u.mas).to(u.kpc)
c = coord.SkyCoord(
    ra=catalog['ra']*u.deg, dec=catalog['dec']*u.deg,
    pm_ra_cosdec=catalog['pmra']*u.mas/u.yr, pm_dec=catalog['pmdec']*u.mas/u.yr,
    radial_velocity=catalog['radial_velocity']*u.km/u.s,
    distance=distance,
    frame='icrs',
)
gal = c.transform_to(coord.Galactic)

In [18]:
# convert from RA, Dec, parallax, pmra, pmdec, radial_velocity to Galactic coordinates
distance = coord.Distance(parallax=catalog['parallax']*u.mas).to(u.kpc)
c = coord.SkyCoord(
    ra=catalog['ra']*u.deg, dec=catalog['dec']*u.deg,
    pm_ra_cosdec=catalog['pmra']*u.mas/u.yr, pm_dec=catalog['pmdec']*u.mas/u.yr,
    radial_velocity=catalog['radial_velocity']*u.km/u.s,
    distance=distance,
    frame='icrs',
)
gal = c.transform_to(coord.Galactic)

cartesian_d = gal.cartesian.differentials['s']
vx = cartesian_d.d_x.to_value(u.km / u.s)
vy = cartesian_d.d_y.to_value(u.km / u.s)
vz = cartesian_d.d_z.to_value(u.km / u.s)
vperp = np.sqrt(vx**2 + vz**2)
vrot = vy + 224.7092

In [22]:
# Calculate 2D vperp and vrot distribution
bins = (1000, 500)
hist_range = ((-500, 500), (0, 500))
# unweighted
C, xedges, yedges = np.histogram2d(vrot, vperp, bins, range=hist_range)

# calculate grid and unit area
X, Y = np.meshgrid(xedges, yedges)
dx = xedges[1] - xedges[0]
dy = yedges[1] - yedges[0]
dA = dx * dy

In [23]:
# Plot the Toomre diagram
fig, ax = plt.subplots(1, figsize=(16, 8))

# plot unweighted Toomre diagram
vmin = 1e0
vmax = 1e3
norm = mpl.colors.LogNorm(vmin=vmin, vmax=vmax, clip=False)
C_new = C.copy() / dA
C_new[C_new < vmin] = 0

im = ax.pcolormesh(X, Y, C_new.T, norm=norm, shading='auto')
ax.set_xlim(-500, 500)
ax.set_ylim(0, 500)
ax.set_xlabel(r'$V_Y$ [km/s]')
ax.set_ylabel(r'$\sqrt{V_X^2 + V_Z^2}$ [km/s]')
ax.set_xticks([-500, -250, 0, 250, 500])
ax.set_yticks([100, 200, 300, 400])

# add colorbar
cbar = plt.colorbar(
    im, ax=ax, label='Counts (km/s)$^{-2}$',
    orientation="horizontal", pad=0.15
)
cbar.ax.xaxis.set_tick_params(which='both', direction='out')

plt.show()
fig.savefig(
    os.path.join(plotdir, 'toomre_diagram.png'), dpi=300, 
    bbox_inches='tight'))